# Instructions
* One vs all logistic regression
* Softmax regression = generalization to handle multiple classes
* Neural network with one hidden layer, and numerically checking the gradient
* Now 2 hidden layers and different activation f'ns, see what performs best
* With best model, do confusion matrix on test set
* 4 page report

In [1]:
# notebook setup
import random 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.decomposition import PCA

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Importing the data

We have 50 000 32x32 images in the train set and a "labels" file with 50 000 lines.

For the images, we'll store them as a 50000x3072 np array, so the first 1024 columns are the value of the red pixel in the image of that row, and the next two 1024 columns are the green and blue values.

In [2]:
def get_data(cifar_dirname="D:\\Unicamp\\MC886\\Git\\T2\\train\\", upperbound=50000):    
    classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    X = np.empty(shape=(1,3072))
    # we load the 50 000 images in X one after the other, one image being 1x3072, to obtain the 50000x3072 array
    for i in range(upperbound):
        # to have an update of where we're at once in a while
        if i % 1000 == 0:
            print(i)
        # open the current image
        current_im  = Image.open(cifar_dirname + str(i).zfill(5) + ".png" )
        # reshape it into an (1,3072) array, so the red values of all the pixels, then green then blue (32*32*3)
        reshaped_im = np.reshape(np.asarray(current_im, 'uint8'), (1,3072))
        # vertical stack the image into X, what will contain all the images
        X           = np.vstack([X, reshaped_im])
        
    # we want the array to be of type unsigned int on 8 bit (between 0 and 255), so that it occupies the minimum space
    X = X.astype("uint8")
    X = np.delete(X, (0), axis=0) # delete the first row that's empty
    return X

In [8]:
X = get_data()
# we load the 50 000 labels for the txt file
y = np.loadtxt("D:\\Unicamp\\MC886\\Git\\T2\\train\\labels")
y.shape
# since loading the images took a lot of time (~3hr), we'll save them in a binary file (.npy)
np.save("D:\\Unicamp\\MC886\\Git\\T2\\images", X)

In [3]:
# now to get the nparray back
X = np.load("../T2/images.npy")
# also, for the y array, like earlier:
y = np.loadtxt("../T2/train/labels")

# One vs all classifiers

We need to do feature scaling on the images before feeding them into the algorithms. We first get the values between -1 and 1: since they're all between 0 and 255, we'll divide by 127 and substract 1.

We then calculate the mean of each image and substract each row by that value.

We then try to reduce the number of feutres by projecting on a principal subspace, with the PCA algorithm. To do so we first run it on the data, then print the variances and decide the number of features we want to keep.

In [ ]:
# standardization of the data
X = np.divide(X, 127).astype("float64")
X -= 1

# calculate the mean of each image
mean = np.mean(X, axis=1) # shape (50000,)
X = (X.transpose() - mean) # substract each row by corresponding mean
X = X.transpose() # transpose X back

# apply PCA
pca = PCA()
pca.fit(X)

In [5]:
for i in range(-10,10):
    a = 0
    for x in pca.explained_variance_:
        if x < pow(10,i):
            a = a + 1
    print("< e^"+str(i)+": "+str(a))

< e^-10: 1
< e^-9: 1
< e^-8: 1
< e^-7: 1
< e^-6: 1
< e^-5: 37
< e^-4: 584
< e^-3: 1365
< e^-2: 2141
< e^-1: 2759
< e^0: 3007
< e^1: 3062
< e^2: 3071
< e^3: 3072
< e^4: 3072
< e^5: 3072
< e^6: 3072
< e^7: 3072
< e^8: 3072
< e^9: 3072


We'll try to keep 2400 features, thus disregarding the ~600 features with a covariance inferiour to 10^-4. This will already help a lot with the calculations.

In [ ]:
pca.n_components = 2400
X_reduced = pca.fit_transform(X)
X_reduced.shape
np.save("../T2/images_reduced.npy", X_reduced)

In [4]:
X_reduced = np.load("../T2/images_reduced.npy")
y = np.loadtxt("../T2/train/labels", dtype=int)

In [5]:
X_reduced[0:10]

array([[  1.16218563e+01,   4.35054736e+00,  -1.06141394e+01, ...,
         -3.85243468e-03,   1.49823087e-02,   1.18771391e-02],
       [  7.61379453e-01,   1.16192626e+01,   1.71488161e+00, ...,
         -4.27453536e-03,  -1.07817977e-02,  -2.84231972e-03],
       [ -2.76356677e+01,   5.13644401e-01,  -1.40725825e+00, ...,
         -5.98168370e-03,  -8.11071692e-03,   3.79169774e-03],
       ..., 
       [  8.69513291e+00,  -6.43107970e+00,  -2.49142396e+00, ...,
          6.53476378e-04,  -5.13104828e-04,  -1.07622296e-02],
       [ -6.48995440e-01,  -8.32589757e-01,   1.58659106e+01, ...,
          8.83920684e-03,  -1.50775800e-02,   1.29620180e-02],
       [ -1.20069510e+01,  -4.86847553e+00,  -1.58816572e+00, ...,
         -2.84157815e-03,   7.87094837e-03,   8.05116643e-04]])

We now train our 10 different logistic regressions, using the one vs all method (one class at 1, the others at 0), and collect our 10 classifiers in a list.

In [9]:
from sklearn.linear_model import LogisticRegression

def classifier_onevsall(data, labels, num_class):
    labels_onevsall = (labels == num_class).astype(int)
    logreg = LogisticRegression()
    logreg.fit(data, labels_onevsall)
    return logreg

classifiers = []
for i in range(10):
    print(i)
    classifiers.append(classifier_onevsall(X_reduced, y, i))

print(classifiers)

0
1
2
3
4
5
6
7
8
9
[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random

We'll now evaluate our models on the training data and test data. We hence first need to import and process the test data, we'll write a f'n for speeding up the preprocessing.

In [10]:
def preprocessing_data(data):
    # get values between -1 and 1
    data = np.divide(data, 127).astype("float64")
    data -= 1
    
    # calculate the mean of each image
    mean = np.mean(data, axis=1) # shape (50000,)
    data = (data.transpose() - mean) # substract each row by corresponding mean
    # note that data is now transposed, shape (3072, 50000), let's put it back
    data = data.transpose()
    
    # apply the PCA algorithm
    pca = PCA(n_components=2400)
    data = pca.fit_transform(data)
    print(data.shape)
    return data

In [12]:
X_test = get_data("D:\\Unicamp\\MC886\\Git\\T2\\test\\", 10000)
np.save("D:\\Unicamp\\MC886\\Git\\T2\\images_test.npy", X_test)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [13]:
X_test_reduced = preprocessing_data(X_test)
np.save("D:\\Unicamp\\MC886\\Git\\T2\\images_test_reduced.npy", X_test_reduced)

(10000, 2400)


In [5]:
X_test = np.load("../T2/images_test_reduced.npy")
y_test = np.loadtxt("../T2/test/labels", dtype=int)

Now we run the models on our test data, and see how well they perform.

In [15]:
def test_onevsall(classifiers, data, nbr_classes=10):
    predictions = np.empty((data.shape[0],nbr_classes))
    for i in range(nbr_classes):
        predictor = classifiers[i]
        # put the predicted values by each classifier for the whole data (shape (nbr_elements,nbr_classes))
        predictions[:, i] = predictor.predict(data)
    # return the indice of highest element (so where 1 is predicted)
    # if more than one classifier returned 1 for that sample, first encountered is kept
    print(predictions.shape)
    pred_indices = np.argmax(predictions, axis=1)
    return pred_indices

In [16]:
train_predictions = test_onevsall(classifiers, X_reduced)

(50000, 10)


In [17]:
test_predictions = test_onevsall(classifiers, X_test_reduced, 10)

(10000, 10)


In [18]:
# now that we have our predictions, we'll calculate the percentage of right answers
print(np.mean(y == train_predictions))
print(np.mean(y_test == test_predictions))

0.2683
0.115


On our train data we do fairly well (26,8%), but on the test data we have very bad results, only 11,5% which is only slightly better than just choosing the same class everytime (10%).

We'll hence try a more powerful model and build a Softmax regression.

In [6]:
class SoftmaxRegression(object):

    """Softmax regression classifier.
    Parameters
    ------------
    eta : float (default: 0.01)
        Learning rate (between 0.0 and 1.0)
    epochs : int (default: 50)
        Passes over the training dataset.
        Prior to each epoch, the dataset is shuffled
        if `minibatches > 1` to prevent cycles in stochastic gradient descent.
    l2 : float
        Regularization parameter for L2 regularization.
        No regularization if l2=0.0.
    minibatches : int (default: 1)
        The number of minibatches for gradient-based optimization.
        If 1: Gradient Descent learning
        If len(y): Stochastic Gradient Descent (SGD) online learning
        If 1 < minibatches < len(y): SGD Minibatch learning
    n_classes : int (default: None)
        A positive integer to declare the number of class labels
        if not all class labels are present in a partial training set.
        Gets the number of class labels automatically if None.
    random_seed : int (default: None)
        Set random state for shuffling and initializing the weights.

    Attributes
    -----------
    w_ : 2d-array, shape={n_features, 1}
      Model weights after fitting.
    b_ : 1d-array, shape={1,}
      Bias unit after fitting.
    cost_ : list
        List of floats, the average cross_entropy for each epoch.

    """
    def __init__(self, eta=0.01, epochs=50,
                 l2=0.0,
                 minibatches=1,
                 n_classes=None,
                 random_seed=None):

        self.eta = eta
        self.epochs = epochs
        self.l2 = l2
        self.minibatches = minibatches
        self.n_classes = n_classes
        self.random_seed = random_seed

    def _fit(self, X, y, init_params=True):
        if init_params:
            if self.n_classes is None:
                self.n_classes = np.max(y) + 1
            self._n_features = X.shape[1]

            self.b_, self.w_ = self._init_params(
                weights_shape=(self._n_features, self.n_classes),
                bias_shape=(self.n_classes,),
                random_seed=self.random_seed)
            self.cost_ = []

        y_enc = self._one_hot(y=y, n_labels=self.n_classes, dtype=np.float)

        for i in range(self.epochs):
            for idx in self._yield_minibatches_idx(
                    n_batches=self.minibatches,
                    data_ary=y,
                    shuffle=True):
                # givens:
                # w_ -> n_feat x n_classes
                # b_  -> n_classes

                # net_input, softmax and diff -> n_samples x n_classes:
                net = self._net_input(X[idx], self.w_, self.b_)
                softm = self._softmax(net)
                diff = softm - y_enc[idx]
                mse = np.mean(diff, axis=0)

                # gradient -> n_features x n_classes
                grad = np.dot(X[idx].T, diff)
                
                # update in opp. direction of the cost gradient
                self.w_ -= (self.eta * grad +
                            self.eta * self.l2 * self.w_)
                self.b_ -= (self.eta * np.sum(diff, axis=0))

            # compute cost of the whole epoch
            net = self._net_input(X, self.w_, self.b_)
            softm = self._softmax(net)
            cross_ent = self._cross_entropy(output=softm, y_target=y_enc)
            cost = self._cost(cross_ent)
            self.cost_.append(cost)
        return self

    def fit(self, X, y, init_params=True):
        """Learn model from training data.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target values.
        init_params : bool (default: True)
            Re-initializes model parametersprior to fitting.
            Set False to continue training with weights from
            a previous model fitting.

        Returns
        -------
        self : object

        """
        if self.random_seed is not None:
            np.random.seed(self.random_seed)
        self._fit(X=X, y=y, init_params=init_params)
        self._is_fitted = True
        return self
    
    def _predict(self, X):
        probas = self.predict_proba(X)
        return self._to_classlabels(probas)
 
    def predict(self, X):
        """Predict targets from X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        ----------
        target_values : array-like, shape = [n_samples]
          Predicted target values.

        """
        if not self._is_fitted:
            raise AttributeError('Model is not fitted, yet.')
        return self._predict(X)

    def predict_proba(self, X):
        """Predict class probabilities of X from the net input.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        ----------
        Class probabilties : array-like, shape= [n_samples, n_classes]

        """
        net = self._net_input(X, self.w_, self.b_)
        softm = self._softmax(net)
        return softm

    def _net_input(self, X, W, b):
        return (X.dot(W) + b)

    def _softmax(self, z):
        return (np.exp(z.T) / np.sum(np.exp(z), axis=1)).T

    def _cross_entropy(self, output, y_target):
        return - np.sum(np.log(output) * (y_target), axis=1)

    def _cost(self, cross_entropy):
        L2_term = self.l2 * np.sum(self.w_ ** 2)
        cross_entropy = cross_entropy + L2_term
        return 0.5 * np.mean(cross_entropy)

    def _to_classlabels(self, z):
        return z.argmax(axis=1)
    
    def _init_params(self, weights_shape, bias_shape=(1,), dtype='float64',
                     scale=0.01, random_seed=None):
        """Initialize weight coefficients."""
        if random_seed:
            np.random.seed(random_seed)
        w = np.random.normal(loc=0.0, scale=scale, size=weights_shape)
        b = np.zeros(shape=bias_shape)
        return b.astype(dtype), w.astype(dtype)
    
    def _one_hot(self, y, n_labels, dtype):
        """Returns a matrix where each sample in y is represented
           as a row, and each column represents the class label in
           the one-hot encoding scheme.

        Example:

            y = np.array([0, 1, 2, 3, 4, 2])
            mc = _BaseMultiClass()
            mc._one_hot(y=y, n_labels=5, dtype='float')

            np.array([[1., 0., 0., 0., 0.],
                      [0., 1., 0., 0., 0.],
                      [0., 0., 1., 0., 0.],
                      [0., 0., 0., 1., 0.],
                      [0., 0., 0., 0., 1.],
                      [0., 0., 1., 0., 0.]])

        """
        mat = np.zeros((len(y), n_labels))
        for i, val in enumerate(y):
            mat[i, val] = 1
        return mat.astype(dtype)    
    
    def _yield_minibatches_idx(self, n_batches, data_ary, shuffle=True):
            indices = np.arange(data_ary.shape[0])

            if shuffle:
                indices = np.random.permutation(indices)
            if n_batches > 1:
                remainder = data_ary.shape[0] % n_batches

                if remainder:
                    minis = np.array_split(indices[:-remainder], n_batches)
                    minis[-1] = np.concatenate((minis[-1],
                                                indices[-remainder:]),
                                               axis=0)
                else:
                    minis = np.array_split(indices, n_batches)

            else:
                minis = (indices,)

            for idx_batch in minis:
                yield idx_batch
    
    def _shuffle_arrays(self, arrays):
        """Shuffle arrays in unison."""
        r = np.random.permutation(len(arrays[0]))
        return [ary[r] for ary in arrays]

In [11]:
softmax = SoftmaxRegression(eta=0.01, epochs=10, minibatches=10, random_seed=42)
softmax.fit(X_reduced, y)
Y_soft_pred = softmax.predict(X_test)

C:\Users\VictorAkiraHassudaSi\Anaconda3\lib\site-packages\ipykernel_launcher.py:166: RuntimeWarning: overflow encountered in exp
C:\Users\VictorAkiraHassudaSi\Anaconda3\lib\site-packages\ipykernel_launcher.py:166: RuntimeWarning: invalid value encountered in true_divide


In [12]:
print(np.mean(y_test == Y_soft_pred))

0.1


Creating a neural network with 1 hidden layer

In [8]:
import tensorflow as tf


config = tf.contrib.learn.RunConfig(tf_random_seed=42) # not shown in the config

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_reduced)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[5000], n_classes=10, feature_columns=feature_cols, config=config)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) # if TensorFlow >= 1.1
dnn_clf.fit(X_reduced, y, batch_size=50, steps=4000)

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000015B71A12E10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\VICTOR~1\\AppData\\Local\\Temp\\tmpxwfitaxd'}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer sup

SKCompat()

In [9]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test)
accuracy_score(y_test, y_pred['classes'])

INFO:tensorflow:Restoring parameters from C:\Users\VICTOR~1\AppData\Local\Temp\tmpxwfitaxd\model.ckpt-4000


0.1807

In [10]:
from sklearn.metrics import log_loss

y_pred_proba = y_pred['probabilities']
log_loss(y_test, y_pred_proba)

3.4715865068787259

Below is the training with 2 hidden layers


In [11]:
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[5000,1000], n_classes=10, feature_columns=feature_cols, config=config)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) # if TensorFlow >= 1.1
dnn_clf.fit(X_reduced, y, batch_size=50, steps=4000)

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000015B39634A90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\VICTOR~1\\AppData\\Local\\Temp\\tmpshwqaiv5'}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer sup

SKCompat()

In [12]:
from sklearn.metrics import accuracy_score

y_pred2 = dnn_clf.predict(X_test)
accuracy_score(y_test, y_pred2['classes'])

INFO:tensorflow:Restoring parameters from C:\Users\VICTOR~1\AppData\Local\Temp\tmpshwqaiv5\model.ckpt-4000


0.17169999999999999

In [13]:
from sklearn.metrics import log_loss

y_pred_proba2 = y_pred2['probabilities']
log_loss(y_test, y_pred_proba2)

3.2922673461669825